In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeClassifier
%matplotlib inline

In [2]:
data = pd.read_csv('datatelecom_churn.csv', sep=',')
data.head()

,State,Account length,Area code,International plan,Voice mail plan,Number vmail messages,Total day minutes,Total day calls,Total day charge,Total eve minutes,Total eve calls,Total eve charge,Total night minutes,Total night calls,Total night charge,Total intl minutes,Total intl calls,Total intl charge,Customer service calls,Churn
0,KS,128,415,No,Yes,25,265.1,110,45.07,197.4,99,16.78,244.7,91,11.01,10.0,3,2.70,1,False
1,OH,107,415,No,Yes,26,161.6,123,27.47,195.5,103,16.62,254.4,103,11.45,13.7,3,3.70,1,False
2,NJ,137,415,No,No,0,243.4,114,41.38,121.2,110,10.30,162.6,104,7.32,12.2,5,3.29,0,False
3,OH,84,408,Yes,No,0,299.4,71,50.90,61.9,88,5.26,196.9,89,8.86,6.6,7,1.78,2,False
4,OK,75,415,Yes,No,0,166.7,113,28.34,148.3,122,12.61,186.9,121,8.41,10.1,3,2.73,3,False


In [3]:
data.drop(['State', 'Voice mail plan'], axis = 1, inplace = True)

In [4]:
data['International plan'] = data['International plan'].map({'Yes': 1, 'No': 0})
data.head()

,Account length,Area code,International plan,Number vmail messages,Total day minutes,Total day calls,Total day charge,Total eve minutes,Total eve calls,Total eve charge,Total night minutes,Total night calls,Total night charge,Total intl minutes,Total intl calls,Total intl charge,Customer service calls,Churn
0,128,415,0,25,265.1,110,45.07,197.4,99,16.78,244.7,91,11.01,10.0,3,2.70,1,False
1,107,415,0,26,161.6,123,27.47,195.5,103,16.62,254.4,103,11.45,13.7,3,3.70,1,False
2,137,415,0,0,243.4,114,41.38,121.2,110,10.30,162.6,104,7.32,12.2,5,3.29,0,False
3,84,408,1,0,299.4,71,50.90,61.9,88,5.26,196.9,89,8.86,6.6,7,1.78,2,False
4,75,415,1,0,166.7,113,28.34,148.3,122,12.61,186.9,121,8.41,10.1,3,2.73,3,False


In [5]:
y = data['Churn'].astype(int)
X = data.drop('Churn', axis = 1)

### Дерева рішень

In [6]:
from sklearn.model_selection import train_test_split, cross_val_score

In [7]:
# Розбиття м-ці X та y на дві частини кожну
X_train, X_valid, y_train, y_valid = train_test_split(X,y, test_size = .3, random_state = 17)

In [8]:
# Сворення екземпляру класифікатора із random_state = 17
first_tree = DecisionTreeClassifier(random_state = 17)

In [9]:
# Середня якість Кросс-валідації(долі правильних відповідей на 30% даних -- тестових) БЕЗ налаштування параметрів
np.mean(cross_val_score(first_tree, X_train, y_train, cv = 5))

0.9138423504976518

### KNN - метод найближчих сусідів

In [10]:
from sklearn.neighbors import KNeighborsClassifier

In [11]:
first_knn = KNeighborsClassifier()

In [12]:
np.mean(cross_val_score(first_knn, X_train, y_train, cv = 5))

0.8671274043984523

### Підбір максимальної глибини методу дерева

In [13]:
from sklearn.model_selection import GridSearchCV

In [14]:
# перебір по всіх параметрах (30 шт)
tree_params = {"max_depth": np.arange(1, 11), "max_features": [0.5, 0.7, 1]}

In [15]:
# n_jobs = -1 --> усі ядра процесора
tree_grid = GridSearchCV(first_tree, tree_params, cv=5, n_jobs=-1)

In [16]:
%%time
tree_grid.fit(X_train, y_train)

Wall time: 2.21 s


GridSearchCV(cv=5, estimator=DecisionTreeClassifier(random_state=17), n_jobs=-1,
             param_grid={'max_depth': array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10]),
                         'max_features': [0.5, 0.7, 1]})

In [17]:
tree_grid.best_score_, tree_grid.best_params_

(0.9391366681677404, {'max_depth': 6, 'max_features': 0.7})

### Підбір максимальної глибини методу найближчих сусідів

In [26]:
knn_params = {'n_neighbors': range(5, 30, 5)}

In [27]:
knn_grid = GridSearchCV(first_knn, knn_params, cv=5, n_jobs=-1)

In [28]:
%%time
knn_grid.fit(X_train, y_train)

Wall time: 209 ms


GridSearchCV(cv=5, estimator=KNeighborsClassifier(), n_jobs=-1,
             param_grid={'n_neighbors': range(5, 30, 5)})

In [30]:
knn_grid.best_score_, knn_grid.best_params_

(0.8701289391697531, {'n_neighbors': 10})

### Взяті оптимальні параметри методу дерева

In [31]:
tree_grid.best_estimator_

DecisionTreeClassifier(max_depth=6, max_features=0.7, random_state=17)

In [32]:
# Прогноз дерева за відкладеною вибіркою:
tree_grid_predict = tree_grid.predict(X_valid);

In [33]:
from sklearn.metrics import accuracy_score

In [34]:
# Перевірка точності Кросс-валідації (перевірка реальних міток та прогнозу)
accuracy_score(y_valid,  tree_grid_predict)

0.936

In [35]:
# Нижня межа точності Кросс-валідації для коректної роботи програми 
1 - np.mean(y)

0.8550855085508551

## Візуалізація даних

In [94]:
import graphviz
from sklearn import tree

In [95]:
# Для Linux:
# !dot -Tpng '../../img/deep_toy_tree.dot' -o '../../img/deep_toy_tree.png'

In [108]:
dot_data = tree.export_graphviz(tree_grid.best_estimator_, out_file='telecom_tree.dot', feature_names = X.columns ,filled = True, rounded=True,special_characters=True)
graph = graphviz.Source(dot_data)

In [111]:
graph;